In [97]:
#check for gpu
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


1) IMPORTS

In [118]:
import numpy as np
import matplotlib.pyplot as plt
from Utils import plt_house_x, plt_contour_wgrad, plt_divergence, plt_gradients
import pandas as pd
import seaborn as sns
import copy,math
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
np.set_printoptions(precision=2)

In [99]:
test_set = pd.read_csv(r'C:\Users\nourm\OneDrive\Desktop\PROJECTS\Food_Delivery_Time_Predictor\FDT_Predictor\Dataset\test.csv')

train_set=pd.read_csv(r'C:\Users\nourm\OneDrive\Desktop\PROJECTS\Food_Delivery_Time_Predictor\FDT_Predictor\Dataset\train.csv')

STEP 2 -EXPLORE AND VISUALISE DATA

In [100]:
print("Dataset shape" , train_set.shape)

print("\nColumn names:" ,train_set.columns.tolist())

Dataset shape (45593, 20)

Column names: ['ID', 'Delivery_person_ID', 'Delivery_person_Age', 'Delivery_person_Ratings', 'Restaurant_latitude', 'Restaurant_longitude', 'Delivery_location_latitude', 'Delivery_location_longitude', 'Order_Date', 'Time_Orderd', 'Time_Order_picked', 'Weatherconditions', 'Road_traffic_density', 'Vehicle_condition', 'Type_of_order', 'Type_of_vehicle', 'multiple_deliveries', 'Festival', 'City', 'Time_taken(min)']


In [101]:
print(train_set.head().to_string())


        ID Delivery_person_ID Delivery_person_Age Delivery_person_Ratings  Restaurant_latitude  Restaurant_longitude  Delivery_location_latitude  Delivery_location_longitude  Order_Date Time_Orderd Time_Order_picked      Weatherconditions Road_traffic_density  Vehicle_condition Type_of_order Type_of_vehicle multiple_deliveries Festival            City Time_taken(min)
0  0x4607     INDORES13DEL02                   37                     4.9            22.745049             75.892471                   22.765049                    75.912471  19-03-2022    11:30:00          11:45:00       conditions Sunny                High                   2        Snack      motorcycle                    0      No           Urban         (min) 24
1  0xb379     BANGRES18DEL02                   34                     4.5            12.913041             77.683237                   13.043041                    77.813237  25-03-2022    19:45:00          19:50:00      conditions Stormy                 Jam  

In [102]:
train_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45593 entries, 0 to 45592
Data columns (total 20 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   ID                           45593 non-null  object 
 1   Delivery_person_ID           45593 non-null  object 
 2   Delivery_person_Age          45593 non-null  object 
 3   Delivery_person_Ratings      45593 non-null  object 
 4   Restaurant_latitude          45593 non-null  float64
 5   Restaurant_longitude         45593 non-null  float64
 6   Delivery_location_latitude   45593 non-null  float64
 7   Delivery_location_longitude  45593 non-null  float64
 8   Order_Date                   45593 non-null  object 
 9   Time_Orderd                  45593 non-null  object 
 10  Time_Order_picked            45593 non-null  object 
 11  Weatherconditions            45593 non-null  object 
 12  Road_traffic_density         45593 non-null  object 
 13  Vehicle_conditio

In [103]:
train_set.describe()

,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Vehicle_condition
count,45593.000000,45593.000000,45593.000000,45593.000000,45593.000000
mean,17.017729,70.231332,17.465186,70.845702,1.023359
std,8.185109,22.883647,7.335122,21.118812,0.839065
min,-30.905562,-88.366217,0.010000,0.010000,0.000000
25%,12.933284,73.170000,12.988453,73.280000,0.000000
50%,18.546947,75.898497,18.633934,76.002574,1.000000
75%,22.728163,78.044095,22.785049,78.107044,2.000000
max,30.914057,88.433452,31.054057,88.563452,3.000000


In [104]:
train_set.isnull().sum()


ID                             0
Delivery_person_ID             0
Delivery_person_Age            0
Delivery_person_Ratings        0
Restaurant_latitude            0
Restaurant_longitude           0
Delivery_location_latitude     0
Delivery_location_longitude    0
Order_Date                     0
Time_Orderd                    0
Time_Order_picked              0
Weatherconditions              0
Road_traffic_density           0
Vehicle_condition              0
Type_of_order                  0
Type_of_vehicle                0
multiple_deliveries            0
Festival                       0
City                           0
Time_taken(min)                0
dtype: int64

In [105]:
train_set.replace({"NaN": np.nan}, regex=True, inplace = True)

Change column types

In [106]:
train_set['Delivery_person_Age']=train_set['Delivery_person_Age'].astype(float)
train_set['Delivery_person_Ratings']=train_set['Delivery_person_Ratings'].astype(float)

In [107]:
# Converting order_date dtype into datetime
train_set['Order_Date']=pd.to_datetime(train_set['Order_Date'])

# Creating three column for day,month and year
train_set['Order_day']=train_set['Order_Date'].dt.day
train_set['Order_month']=train_set['Order_Date'].dt.month
train_set['Order_year']=train_set['Order_Date'].dt.year

C:\Users\nourm\AppData\Local\Temp\ipykernel_23352\3334419967.py:2: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  train_set['Order_Date']=pd.to_datetime(train_set['Order_Date'])


In [108]:
# Shifting columns
col = train_set.pop("Order_day")
train_set.insert(10,"Order_day",col)

col = train_set.pop("Order_month")
train_set.insert(11,"Order_month",col)

col = train_set.pop("Order_year")
train_set.insert(12,"Order_year",col)

In [109]:
# Converting time_order dtype into datetime
train_set['Time_Orderd']=pd.to_datetime(train_set['Time_Orderd'])

# Creating two new column for hour and minute
train_set['Hour_order']=train_set['Time_Orderd'].dt.hour
train_set['Min_order']=train_set['Time_Orderd'].dt.minute

C:\Users\nourm\AppData\Local\Temp\ipykernel_23352\3487480426.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  train_set['Time_Orderd']=pd.to_datetime(train_set['Time_Orderd'])


In [110]:
# Now Shifting columns
col = train_set.pop("Hour_order")
train_set.insert(14,"Hour_order",col)

col = train_set.pop("Min_order")
train_set.insert(15,"Min_order",col)

In [111]:
# Converting time_order_picked dtype into datetime
train_set['Time_Order_picked']=pd.to_datetime(train_set['Time_Order_picked'])

# Creating two new column for hour and minute
train_set['Hour_order_picked']=train_set['Time_Order_picked'].dt.hour
train_set['Min_order_picked']=train_set['Time_Order_picked'].dt.minute

C:\Users\nourm\AppData\Local\Temp\ipykernel_23352\3827799642.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  train_set['Time_Order_picked']=pd.to_datetime(train_set['Time_Order_picked'])


In [112]:
# Now Shifting columns
col = train_set.pop("Hour_order_picked")
train_set.insert(17,"Hour_order_picked",col)

col = train_set.pop("Min_order_picked")
train_set.insert(18,"Min_order_picked",col)

STEP 3- CLEAN DATA + FEATURE ENGINEERING

In [113]:
# Shows how many missing values in each column
print(train_set.isnull().sum())

ID                                0
Delivery_person_ID                0
Delivery_person_Age            1854
Delivery_person_Ratings        1908
Restaurant_latitude               0
Restaurant_longitude              0
Delivery_location_latitude        0
Delivery_location_longitude       0
Order_Date                        0
Time_Orderd                    1731
Order_day                         0
Order_month                       0
Order_year                        0
Time_Order_picked                 0
Hour_order                     1731
Min_order                      1731
Weatherconditions               616
Hour_order_picked                 0
Min_order_picked                  0
Road_traffic_density            601
Vehicle_condition                 0
Type_of_order                     0
Type_of_vehicle                   0
multiple_deliveries             993
Festival                        228
City                           1200
Time_taken(min)                   0
dtype: int64


In [114]:
train_set['Delivery_person_Age'] = train_set['Delivery_person_Age'].fillna(train_set.Delivery_person_Age.median())
train_set['Delivery_person_Ratings'] = train_set['Delivery_person_Ratings'].fillna(train_set.Delivery_person_Ratings.median())
train_set['Time_Orderd'] = train_set['Time_Orderd'].fillna(train_set.Time_Orderd.mode()[0])
train_set['Hour_order'] = train_set['Hour_order'].fillna(train_set.Hour_order.mode()[0])
train_set['Min_order'] = train_set['Min_order'].fillna(train_set.Min_order.mode()[0])
train_set['Weatherconditions'] = train_set['Weatherconditions'].fillna(train_set.Weatherconditions.mode()[0])
train_set['Road_traffic_density'] = train_set['Road_traffic_density'].fillna(train_set.Road_traffic_density.mode()[0])
train_set['multiple_deliveries'] = train_set['multiple_deliveries'].fillna(train_set.multiple_deliveries.mode()[0])
train_set['Festival'] = train_set['Festival'].fillna(train_set.Festival.mode()[0])
train_set['City'] = train_set['City'].fillna(train_set.City.mode()[0])

In [117]:
train_set.isnull().sum()

ID                             0
Delivery_person_ID             0
Delivery_person_Age            0
Delivery_person_Ratings        0
Restaurant_latitude            0
Restaurant_longitude           0
Delivery_location_latitude     0
Delivery_location_longitude    0
Order_Date                     0
Time_Orderd                    0
Order_day                      0
Order_month                    0
Order_year                     0
Time_Order_picked              0
Hour_order                     0
Min_order                      0
Weatherconditions              0
Hour_order_picked              0
Min_order_picked               0
Road_traffic_density           0
Vehicle_condition              0
Type_of_order                  0
Type_of_vehicle                0
multiple_deliveries            0
Festival                       0
City                           0
Time_taken(min)                0
dtype: int64

label encoding of categorical features


In [119]:

#create label encoders

weather_encoder = LabelEncoder()
traffic_encoder = LabelEncoder()
festival_encoder = LabelEncoder()
city_encoder = LabelEncoder()

#transform labels into encoded data

train_set['Weather_encoded'] = weather_encoder.fit_transform(train_set['Weatherconditions'])
train_set['Traffic_encoded'] = traffic_encoder.fit_transform(train_set['Road_traffic_density'])
train_set['Festival_encoded'] = festival_encoder.fit_transform(train_set['Festival'])
train_set['City_encoded'] = city_encoder.fit_transform(train_set['City'])

In [122]:
#select numerical features for regression- all features except time_taken(min)
features = [
    'Delivery_person_Age', 'Delivery_person_Ratings', 'Restaurant_latitude',
    'Restaurant_longitude', 'Delivery_location_latitude', 'Delivery_location_longitude',
    'Order_day', 'Order_month', 'Hour_order', 'Min_order',
    'Hour_order_picked', 'Min_order_picked', 'Weather_encoded', 
    'Traffic_encoded', 'multiple_deliveries', 'Festival_encoded', 'City_encoded'
]

In [124]:
#prep dataset

x_train=train_set[features].values
y_train = train_set['Time_taken(min)'].values


STEP 3 - APPLY COST FUNCTION AND GRADIENT DESCENT

STEP 4 -TRAIN AND FIT MODEL

STEP 5 -TEST ON TEST DATA